# Log into ArcGIS Online

In [1]:
from arcgis.gis import GIS
import os
import pandas as pd

In [4]:
gis = GIS("http://slustl.maps.arcgis.com/home", "gregbrunner_slugis", "100%BlueGatorade")

# Let's generate a list of Excel Files

In [5]:
xlsx_files = []
for file in os.listdir():
    if 'xlsx'  in file:
        xlsx_files.append(file)
print(xlsx_files)

['fsi-2006.xlsx', 'fsi-2007.xlsx', 'fsi-2008.xlsx', 'fsi-2009.xlsx', 'fsi-2010.xlsx', 'fsi-2011.xlsx', 'fsi-2012.xlsx', 'fsi-2013.xlsx', 'fsi-2014.xlsx', 'fsi-2015.xlsx', 'fsi-2016.xlsx', 'fsi-2017.xlsx', 'fsi-2018.xlsx']


# Let's create a Map

In [6]:
world_map = gis.map()
world_map.basemap = 'dark-gray'
world_map

MapView(basemaps=['dark-gray', 'dark-gray-vector', 'gray', 'gray-vector', 'hybrid', 'national-geographic', 'oc…

# For every Excel file

## 1. Create a DataFrame

## 2. Convert the DataFrame to a CSV

## 3. Publish the CSV

## 4. Add the Published Layer to a Map and Render on the *Total* field

In [7]:
for xl in xlsx_files:

    df = pd.read_excel(xl)
        
    csv = xl[0:-5].upper() + '.csv'
    
    df.to_csv(csv)
    print('Publishing ' + csv)
    
    csv_content = gis.content.add({}, csv)
    layers = csv_content.publish(None, {"Address":"Country"})
    world_map.add_layer(layers, {"renderer":"ClassedSizeRenderer", "field_name":"Total"})
    print('Done ' + csv)
print("Done")

Publishing FSI-2006.csv
Done FSI-2006.csv
Publishing FSI-2007.csv
Done FSI-2007.csv
Publishing FSI-2008.csv
Done FSI-2008.csv
Publishing FSI-2009.csv
Done FSI-2009.csv
Publishing FSI-2010.csv
Done FSI-2010.csv
Publishing FSI-2011.csv
Done FSI-2011.csv
Publishing FSI-2012.csv
Done FSI-2012.csv
Publishing FSI-2013.csv
Done FSI-2013.csv
Publishing FSI-2014.csv
Done FSI-2014.csv
Publishing FSI-2015.csv
Done FSI-2015.csv
Publishing FSI-2016.csv
Done FSI-2016.csv
Publishing FSI-2017.csv
Done FSI-2017.csv
Publishing FSI-2018.csv
Done FSI-2018.csv
Done


# Set the zoom on the map

In [8]:
world_map.center=[0,0]
world_map.zoom = 2

# Save the Map as a Webmap

In [7]:
webmap_properties = {'title':'FSI Project - Greg B',
                     'snippet':'Webmap',
                     'tags':['FSI']}
world_map.save(webmap_properties)
print("Done")

Done


# Applying the Mann-Kendall

## For each excell spreadsheet...

### 1. Read it inot a DataFrame

### 2. Add the year to each column in the DataFrame

### 3. Merge (read *join*) the DataFrame to all previously read DataFrames

In [9]:
init_df = pd.read_excel(xlsx_files[0]).loc[:,'Country']
val_list = []

for xl in xlsx_files:
    print('year is ' + str(xl[4:8]))
    val_list.append('Total'+ str(xl[4:8]))

    df = pd.read_excel(xl)
    new_df = df.loc[:, 'Country':'Total']
    new_df.rename(columns=lambda x: x+str(xl[4:8]), inplace=True)
    try:
        init_df = pd.merge(init_df.to_frame(), new_df, how = 'left', left_on='Country', right_on='Country'+str(xl[4:8]))
    except:
        init_df = pd.merge(init_df, new_df, how = 'left', left_on='Country', right_on='Country'+str(xl[4:8]))
        
print("Done")

year is 2006
year is 2007
year is 2008
year is 2009
year is 2010
year is 2011
year is 2012
year is 2013
year is 2014
year is 2015
year is 2016
year is 2017
year is 2018
Done


# View the DataFrame

In [10]:
init_df.head()

,Country,Country2006,Year2006,Rank2006,Total2006,Country2007,Year2007,Rank2007,Total2007,Country2008,...,Rank2016,Total2016,Country2017,Year2017,Rank2017,Total2017,Country2018,Year2018,Rank2018,Total2018
0,Sudan,Sudan,2006-01-01,1st,112.3,Sudan,2007-01-01,1st,113.7,Sudan,...,4th,111.5,Sudan,2017-01-01,5th,110.6,Sudan,2018-01-01,7th,108.663194
1,Congo Democratic Republic,Congo Democratic Republic,2006-01-01,2nd,110.1,Congo Democratic Republic,2007-01-01,7th,105.5,Congo Democratic Republic,...,8th,110.0,Congo Democratic Republic,2017-01-01,7th,110.0,Congo Democratic Republic,2018-01-01,6th,110.670365
2,Cote d'Ivoire,Cote d'Ivoire,2006-01-01,3rd,109.2,Cote d'Ivoire,2007-01-01,6th,107.3,Cote d'Ivoire,...,21st,97.9,Cote d'Ivoire,2017-01-01,21st,96.5,Cote d'Ivoire,2018-01-01,25th,94.561519
3,Iraq,Iraq,2006-01-01,4th,109.0,Iraq,2007-01-01,2nd,111.4,Iraq,...,11th,104.7,Iraq,2017-01-01,10th,105.4,Iraq,2018-01-01,11th,102.205685
4,Zimbabwe,Zimbabwe,2006-01-01,5th,108.9,Zimbabwe,2007-01-01,4th,110.1,Zimbabwe,...,16th,100.5,Zimbabwe,2017-01-01,13th,101.6,Zimbabwe,2018-01-01,10th,102.293753


# Import the *mk_test* tool

In [11]:
import mk_test

# For every row (*counrty*) in the DataFrame, run the Mann-Kendall Test

In [13]:
init_df['MK Val']= 'No Data'
for idx, row in enumerate(init_df.iterrows()):
    mk_trend_vals = []
    for val in val_list:
        #print(init_df.at[idx,'Country'])
        mk_trend_vals.append(init_df.at[idx,val])

    try:
        print(init_df.at[idx, 'Country'])
        print(mk_trend_vals)
        print(mk_test.mk_test(mk_trend_vals)[0])
        init_df.at[idx,'MK Val']  = mk_test.mk_test(mk_trend_vals)[0]
    except:
        print('ERROR')
        

Sudan
[112.3, 113.7, 113.0, 112.4, 111.8, 108.7, 109.4, 111.0, 110.1, 110.8, 111.5, 110.6, 108.66319439644046]
decreasing
Congo Democratic Republic
[110.1, 105.5, 106.7, 108.7, 109.9, 108.2, 111.2, 111.9, 110.2, 109.7, 110.0, 110.0, 110.67036484651267]
no trend
Cote d'Ivoire
[109.2, 107.3, 104.6, 102.5, 101.2, 102.8, 103.6, 103.5, 101.7, 100.1, 97.9, 96.5, 94.56151916975632]
decreasing
Iraq
[109.0, 111.4, 110.6, 108.6, 107.3, 104.8, 104.3, 103.9, 102.2, 104.4, 104.7, 105.4, 102.20568542153825]
decreasing
Zimbabwe
[108.9, 110.1, 112.5, 114.0, 110.2, 107.9, 106.3, 105.2, 102.8, 100.0, 100.5, 101.6, 102.29375253209211]
decreasing
Chad
[105.9, 108.8, 110.9, 112.2, 113.3, 110.3, 107.6, 109.0, 108.7, 108.4, 110.1, 109.4, 108.31187045747872]
no trend
Somalia
[105.9, 111.1, 114.2, 114.7, 114.3, 113.4, 114.9, 113.9, 112.6, 114.0, 114.0, 113.4, 113.20079608219345]
no trend
Haiti
[104.6, 100.9, 99.3, 101.8, 101.6, 108.0, 104.9, 105.8, 104.3, 104.5, 105.1, 105.3, 101.96955214756449]
no trend
Pakis

Greece 
[41.1, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
ERROR
Argentina
[40.8, 41.4, 41.4, 44.7, 45.8, 46.8, 46.5, 46.1, 47.3, 47.6, 48.4, 48.2, 46.06992256962885]
increasing
South Korea
[39.9, 39.7, 40.6, 41.6, 41.3, 38.8, 37.6, 35.4, 36.4, 36.3, 36.1, 38.1, 35.73391458947191]
decreasing
Germany 
[39.7, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
ERROR
Spain 
[37.4, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
ERROR
Slovenia 
[36.8, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
ERROR
Italy 
[35.1, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
ERROR
United States
[34.5, 33.6, 32.8, 34.0, 35.3, 34.8, 34.8, 33.5, 35.4, 35.4, 34.0, 35.6, 37.7]
increasing
France
[34.3, 34.1, 34.8, 35.3, 34.9, 34.0, 33.6, 32.6, 34.8, 33.7, 34.5, 33.5, 32.22701109659329]
no trend
United Kingdom
[34.2, 34.1, 32.9, 33.6, 33.9, 34.1, 35.3, 33.2, 34.3, 33.4, 32.4, 33.2, 34.29882224337588]
no trend
Portugal 
[32.7, nan, nan, nan, nan, n

C:\Users\greg6750\Documents\IPython Notebooks\Advanced_Python_for_GIS_and_RS\Project 2\mk_test.py:40: RuntimeWarning: invalid value encountered in sign
  s += np.sign(x[j] - x[k])


# Filter out "No Data"

In [19]:
trend_df = init_df[init_df['MK Val']!= 'No Data']

# Reset the Index so that it Publishes easily

In [20]:
trend_df.reset_index()

,index,Country,Country2006,Year2006,Rank2006,Total2006,Country2007,Year2007,Rank2007,Total2007,...,Total2016,Country2017,Year2017,Rank2017,Total2017,Country2018,Year2018,Rank2018,Total2018,MK Val
0,0,Sudan,Sudan,2006-01-01,1st,112.3,Sudan,2007-01-01,1st,113.7,...,111.5,Sudan,2017-01-01,5th,110.6,Sudan,2018-01-01,7th,108.663194,decreasing
1,1,Congo Democratic Republic,Congo Democratic Republic,2006-01-01,2nd,110.1,Congo Democratic Republic,2007-01-01,7th,105.5,...,110.0,Congo Democratic Republic,2017-01-01,7th,110.0,Congo Democratic Republic,2018-01-01,6th,110.670365,no trend
2,2,Cote d'Ivoire,Cote d'Ivoire,2006-01-01,3rd,109.2,Cote d'Ivoire,2007-01-01,6th,107.3,...,97.9,Cote d'Ivoire,2017-01-01,21st,96.5,Cote d'Ivoire,2018-01-01,25th,94.561519,decreasing
3,3,Iraq,Iraq,2006-01-01,4th,109.0,Iraq,2007-01-01,2nd,111.4,...,104.7,Iraq,2017-01-01,10th,105.4,Iraq,2018-01-01,11th,102.205685,decreasing
4,4,Zimbabwe,Zimbabwe,2006-01-01,5th,108.9,Zimbabwe,2007-01-01,4th,110.1,...,100.5,Zimbabwe,2017-01-01,13th,101.6,Zimbabwe,2018-01-01,10th,102.293753,decreasing
5,5,Chad,Chad,2006-01-01,6th,105.9,Chad,2007-01-01,5th,108.8,...,110.1,Chad,2017-01-01,8th,109.4,Chad,2018-01-01,8th,108.311870,no trend
6,6,Somalia,Somalia,2006-01-01,6th,105.9,Somalia,2007-01-01,3rd,111.1,...,114.0,Somalia,2017-01-01,2nd,113.4,Somalia,2018-01-01,2nd,113.200796,no trend
7,7,Haiti,Haiti,2006-01-01,8th,104.6,Haiti,2007-01-01,11th,100.9,...,105.1,Haiti,2017-01-01,11th,105.3,Haiti,2018-01-01,12th,101.969552,no trend
8,8,Pakistan,Pakistan,2006-01-01,9th,103.1,Pakistan,2007-01-01,12th,100.1,...,101.7,Pakistan,2017-01-01,17th,98.9,Pakistan,2018-01-01,20th,96.338060,no trend
9,9,Afghanistan,Afghanistan,2006-01-01,10th,99.8,Afghanistan,2007-01-01,8th,102.3,...,107.9,Afghanistan,2017-01-01,9th,107.3,Afghanistan,2018-01-01,9th,106.620768,no trend


# Publish the MK Result to ArcGIS Online

In [22]:
trend_csv = 'trend_csv.csv'
trend_df.to_csv(trend_csv)
print('Publishing ' + trend_csv)

trend_csv_content = gis.content.add({}, trend_csv)
trend_layers = trend_csv_content.publish(None, {"Address":"Country"})
print('Done.')

Publishing trend_csv.csv
